# Maximum Likelihood

## 1.What is maximum likelihood

Maximum likelihood estimation (MLE) is a **method** to estimate econometric models. MLE first assumes a parametric distribution of y|x, and then maximize the observed joint probability density function with respect to the parameters.

As an example, let's assume a simple linear regression model where

$$y=\beta_0 + \beta_1 x + u$$

We first assume that conditional on X=x, y satisfies a normal distribution with mean equal to $\beta_0 + \beta_1 x$, and variance equal to $\sigma^2$. In this setup, we are using the same assumptions as OLS (although this is not necessary).

With the assumptions, we can explicitly write down the probability density function of one observation 

$$ f(y_i|x_i) = \phi(\frac{y_i-\beta_0-\beta_1x}{\sigma}) $$

And with Gauss-Markov assumption 3 - $u_i$ are independently distributed. We have the joint probability (likelihood) function of all observations as

$$\Pi_i f(y_i|x_i)$$

And we maximize this joint probability with respect to all the parameters

$$\max_{\beta_0, \beta_1, \sigma} \Pi_i f(y_i|x_i)$$

There is one more adjustment we can make to simplify the problem.

Because 1) a maximizing a production is more difficult than maximizing a summation, and 2) optima preserves under strictly increasing transformation of the objective function, we take logarithm of the objective function and solve the following log-likelihood maximization problem

$$\max_{\beta_0, \beta_1, \sigma} \Sigma_i lnf(y_i|x_i)$$

> Advantages of maximimum likelihood estimation\
**More flexible** - can be used with any distribution and any functional form\
**Easy to implement** when you known the probability of y given x

## 2.Another Example - Probit

Assume we want to estimate the effect of hours of study on the odds of getting an A in ECO301.

1. Specify a linear regression model
2. Is there any issue with this linear regression model? 
3. How can you restrict the value of $\hat{y}$ to be between 0 and 1?

A probit model assumes
$$P(y=1) = \Phi(x^T\beta)$$
and 
$$P(y=0) = 1-\Phi(x^T\beta)$$

therefore the likelihood function $f(y_1, y_2,...\mid X, \beta)$ can be written as

$$\Pi \Phi_i(x_i^T\beta)\mathcal{1}(y=1)\times(1-\Phi_i(x_i^T\beta))\mathcal{1}(y=0)$$

where $\Phi()$ is normal cumulative density function. By taking the logarithm, the objective function to be maximized is

$$\sum_i ln\Phi_i(x_i^T\beta)\mathcal{1}(y=1)+ln\Phi_i(-x_i^T\beta)\mathcal{1}(y=0)$$

## 3.Implement Probit model in Python

Here we provide a new source of data - sample datasets from the statsmodels package.You can find a full list of sample datasets [here](https://www.statsmodels.org/stable/datasets/index.html). The spector dataset contains experimental data on the effectiveness of the personalized system of instruction (PSI) program.

Grade - binary variable indicating whether or not a student's gradeimproved.  1 indicates an improvement. \
TUCE  - Test score on economics test \
PSI   - participation in program \
GPA   - Student's grade point average 

In [1]:
import wooldridge as woo
import statsmodels.api as sm

ds = sm.datasets.spector.load_pandas()

In [2]:
df = ds.data
df.head()

,GPA,TUCE,PSI,GRADE
0,2.66,20.0,0.0,0.0
1,2.89,22.0,0.0,0.0
2,3.28,24.0,0.0,0.0
3,2.92,12.0,0.0,0.0
4,4.00,21.0,0.0,1.0


In [3]:
import statsmodels.formula.api as smf

reg = smf.probit("GRADE~GPA+TUCE+PSI",data=df)
res = reg.fit()
res.summary()

Optimization terminated successfully.
         Current function value: 0.400588
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                  GRADE   No. Observations:                   32
Model:                         Probit   Df Residuals:                       28
Method:                           MLE   Df Model:                            3
Date:                Tue, 26 Apr 2022   Pseudo R-squ.:                  0.3775
Time:                        15:24:46   Log-Likelihood:                -12.819
converged:                       True   LL-Null:                       -20.592
Covariance Type:            nonrobust   LLR p-value:                  0.001405
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -7.4523      2.542     -2.931      0.003     -12.435      -2.469
GPA            1.6258      0.694      2.343      0.019       0.266       2.986
TUCE           0.0517      0.084      0.617      0.537      -0.113       0.216
PSI            1.4263      0.595      2.397      0.017       0.260       2.593
==============================================================================
"""

### Interpretations

**Pseudo R-squared** \
Recall that R-squared is a statistic generated in ordinary least squares (OLS) regression that is often used as a goodness-of-fit measure.  In OLS,

$$R^2 = SSR/SST = 1- SSE/SST$$

There are several approaches to thinking about R-squared in OLS:

1. R-squared as explained variability
2. R-squared as improvement from null model to fitted model. The denominator of the ratio can be thought of as the sum of squared errors from the null model – a model predicting the dependent variable without any independent variables. 
3. R-squared as the square of the correlation. R-squared is the square of the correlation between the model’s predicted values and the actual values.

When analyzing data with a maximum likelihood method, an equivalent statistic that has all of the above meanings does not exist. Hence several pseudo R-squareds have been developed to assess some aspect of the model.

For example, statsmodels reports the McFadden's pseudo-R-squared.
$$R^2 = 1-\frac{lnL_{full}}{lnL_{intercept}}$$



The log likelihood of the intercept model is treated as a total sum of squares, and the log likelihood of the full model is treated as the sum of squared errors. So this pseudo R-squared measures the improvement from the intercept-only model to the full model.

**Likelihood Ratio Test (LR) - MLE version of F test**

LR-test utilizes the fact that $-2(lnL_{intercept} - lnL_{full}) \sim \chi^2(k)$ if $\beta_1=\beta_2=...=0$. 

**coefficient**

Interpretation of the coefficients in probit regression is not as straightforward as the interpretations of coefficients in linear regression. 

$$P(y=1) = \Phi(x^T\beta)$$

So,

$$\frac{\partial P(y=1)}{\partial\ x_1} = \Phi(x^T\beta)\times \beta_1$$

The marginal effect of $x_1$ is not a constant.

However, there are limited ways in which we can interpret the individual regression coefficients. 

1. A positive coefficient means that an increase in the predictor leads to an increase in the predicted probability.  A negative coefficient means that an increase in the predictor leads to a decrease in the predicted probability.
2. **Average Marginal Effect** - We can first calculate the marginal effect for each observation and then calculate the average of all marginal effects.
3. **Marginal Effect at the mean** - Alternatively, we can first calculate $\bar{x}$ and substitutes its value into $\Phi(.)$.

### Exercise

Load the **ncaa_rpi.csv** dataset from Coursesite. This dataset contains data on NCAA men's basketball teams. 
We are interestted in whether, once the previous year's post-season RPI (*postrpi_1*) is controlled for, does the pre-season RPI (*prerpi*) - which is supposed to add information on recruiting and player development - help to predict performance. 

Run a probit model using *tourney* - whether the team makes it to the NCAA tournament - as the dependent variable and *prerpi* as the independent variable of interest. Control for previous year's post-season RPI (*postrpi_1*), coach experience (*coachexper*), and winning percentage (*winperc*).

In [4]:
import pandas as pd

df = pd.read_csv("ncaa_rpi.csv")
df.head()

,team,year,conference,postrpi,prerpi,postrpi_1,postrpi_2,recruitrank,wins,losses,winperc,tourney,coachexper,power5
0,Boston College,2003-2004,ACC,19,37,44,55,97,24,10,70.58824,1,23,1
1,Boston College,2009-2010,ACC,115,53,67,131,300,15,16,48.38710,0,27,1
2,Boston College,2012-2013,ACC,114,223,244,65,69,16,17,48.48485,0,28,1
3,Boston College,2015-2016,ACC,249,102,161,204,69,7,25,21.87500,0,25,1
4,Clemson,2003-2004,ACC,104,90,125,176,88,10,18,35.71429,0,28,1


In [5]:
import statsmodels.formula.api as smf

formula = "tourney ~ prerpi + postrpi_1 + coachexper"
reg_ols = smf.ols(formula, data=df)
res_ols = reg_ols.fit()
res_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                tourney   R-squared:                       0.218
Model:                            OLS   Adj. R-squared:                  0.211
Method:                 Least Squares   F-statistic:                     30.78
Date:                Tue, 26 Apr 2022   Prob (F-statistic):           1.40e-17
Time:                        15:24:55   Log-Likelihood:                -195.61
No. Observations:                 336   AIC:                             399.2
Df Residuals:                     332   BIC:                             414.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4865      0.079      6.137      0.000       0.331       0.642
prerpi         0.0002      0.002      0.143      0.886      -0.003       0.003
postrpi_1     -0.0031      0.001     -2.365      0.019      -0.006      -0.001
coachexper     0.0067      0.003      2.486      0.013       0.001       0.012
==============================================================================
Omnibus:                      209.612   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               22.618
Skew:                           0.187   Prob(JB):                     1.23e-05
Kurtosis:                       1.785   Cond. No.                         503.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
reg_probit = smf.probit(formula, data=df)
res_probit = reg_probit.fit()
res_probit.summary()

Optimization terminated successfully.
         Current function value: 0.543789
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                tourney   No. Observations:                  336
Model:                         Probit   Df Residuals:                      332
Method:                           MLE   Df Model:                            3
Date:                Tue, 26 Apr 2022   Pseudo R-squ.:                  0.1914
Time:                        15:25:21   Log-Likelihood:                -182.71
converged:                       True   LL-Null:                       -225.97
Covariance Type:            nonrobust   LLR p-value:                 1.230e-18
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0463      0.250      0.185      0.853      -0.444       0.536
prerpi        -0.0038      0.006     -0.663      0.507      -0.015       0.008
postrpi_1     -0.0071      0.005     -1.462      0.144      -0.017       0.002
coachexper     0.0198      0.009      2.304      0.021       0.003       0.037
==============================================================================
"""

In [6]:
reg_logit = smf.logit(formula, data=df)
res_logit = reg_logit.fit()
res_logit.summary()

Optimization terminated successfully.
         Current function value: 0.540474
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                tourney   No. Observations:                  336
Model:                          Logit   Df Residuals:                      332
Method:                           MLE   Df Model:                            3
Date:                Thu, 21 Apr 2022   Pseudo R-squ.:                  0.1964
Time:                        12:53:27   Log-Likelihood:                -181.60
converged:                       True   LL-Null:                       -225.97
Covariance Type:            nonrobust   LLR p-value:                 4.087e-19
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2228      0.433      0.515      0.607      -0.625       1.071
prerpi        -0.0098      0.011     -0.925      0.355      -0.030       0.011
postrpi_1     -0.0104      0.009     -1.218      0.223      -0.027       0.006
coachexper     0.0300      0.015      2.059      0.040       0.001       0.059
==============================================================================
"""

## 4.More flexible Python implementations

### Use Generic MLE for Probit Model

### Prepare the data

In [10]:
import numpy as np
from scipy import stats
from statsmodels.base.model import GenericLikelihoodModel

In [11]:
ds = sm.datasets.spector.load_pandas()
X = sm.add_constant(data.exog)
y = data.endog 

NameError: name 'data' is not defined

### Build Objective function

To provide your own objective function, you simply need to **overwrite** the `loglike` method.

In [12]:
import statsmodels.api as sm

In [13]:
endog = df["tourney"]
exog = df[["prerpi","postrpi_1","coachexper"]]
exog = sm.add_constant(exog)

/Users/benx/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [14]:
class MyProbit(GenericLikelihoodModel):
    def loglike(self, params): # 2 inputs: data, parameters
        X = self.exog
        y = self.endog
        q = 2 * y - 1
        return stats.norm.logcdf(X@params*q).sum() # returns the objective function

MyProbit is a self-defined class that inherit the `GenericLikelihoodModel`.

### Estimate

In [15]:
res = MyProbit(endog=endog, exog=exog).fit()

Optimization terminated successfully.
         Current function value: 0.549675
         Iterations: 125
         Function evaluations: 222


In [16]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               MyProbit Results                               
==============================================================================
Dep. Variable:                tourney   Log-Likelihood:                -184.69
Model:                       MyProbit   AIC:                             377.4
Method:            Maximum Likelihood   BIC:                             392.7
Date:                Tue, 26 Apr 2022                                         
Time:                        15:27:15                                         
No. Observations:                 336                                         
Df Residuals:                     332                                         
Df Model:                           3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5439      0.252      2.162      0.031       0.051       1.037
prerpi        -0.0047      0.006     -0.801      0.423      -0.016       0.007
postrpi_1     -0.0075      0.005     -1.535      0.125      -0.017       0.002
coachexper     0.0052      0.009      0.611      0.541      -0.011       0.022
==============================================================================
"""

### 5.Extra: Optimization in Python

In Python, we can easily find the local minimum of a scalar function with multiple variables using **scipy**.

In [17]:
from scipy.optimize import minimize

In [18]:
from scipy.optimize import minimize_scalar

In [11]:
def func(x):
    return 2*x**2+x+2

In [12]:
minimize_scalar(func)

     fun: 1.875
    nfev: 28
     nit: 24
 success: True
       x: -0.2499999925800001

As a starting point, we need to define the objective function. This function should take the target parameters as its first argument, and there can be other arguments as well. 

Let's copy and past the `loglike` function and detach it from the MyProbit class by replacing the `self` argument with data. In addition, we add a `-` sign to the return value, so that we convert the maximization problem to a minimization.

In [19]:
def loglike(params, data): # 2 inputs: data, parameters
        X = sm.add_constant(data.exog)
        y = data.endog
        q = 2 * y - 1
        return -stats.norm.logcdf(X@params*q).sum() # returns the objective function

In [21]:
res = minimize(loglike, x0=np.array([0,0,0,0]), args=(ds))
# args provide other arugments to the objective function

/Users/benx/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [22]:
res.x

array([-7.45231082,  1.62581032,  0.05172852,  1.42633244])